In [14]:
from datasets import load_dataset

ds = load_dataset("cfilt/iitb-english-hindi")

d:\Documents\Deep_Learning\Generative-Adversarial-Networks\Transformers\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\Documents\Deep_Learning\Generative-Adversarial-Networks\Transformers\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\disha\.cache\huggingface\hub\datasets--cfilt--iitb-english-hindi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to 

In [ ]:
ds

DatasetDict({
    train: Dataset({
        features: ['translation'],
        num_rows: 1659083
    })
    validation: Dataset({
        features: ['translation'],
        num_rows: 520
    })
    test: Dataset({
        features: ['translation'],
        num_rows: 2507
    })
})

In [23]:
ds['train']

Dataset({
    features: ['translation'],
    num_rows: 1659083
})

In [ ]:
def extract_texts(dataset, lang1="en", lang2="hi"):
    src = [item[lang1] for item in dataset["translation"]]
    tgt = [item[lang2] for item in dataset["translation"]]
    return src, tgt

train_src, train_tgt = extract_texts(ds['train'])
val_src, val_tgt = extract_texts(ds['validation'])


dict_values([[{'en': 'Give your application an accessibility workout', 'hi': 'अपने अनुप्रयोग को पहुंचनीयता व्यायाम का लाभ दें'}, {'en': 'Accerciser Accessibility Explorer', 'hi': 'एक्सेर्साइसर पहुंचनीयता अन्वेषक'}, {'en': 'The default plugin layout for the bottom panel', 'hi': 'निचले पटल के लिए डिफोल्ट प्लग-इन खाका'}, {'en': 'The default plugin layout for the top panel', 'hi': 'ऊपरी पटल के लिए डिफोल्ट प्लग-इन खाका'}, {'en': 'A list of plugins that are disabled by default', 'hi': 'उन प्लग-इनों की सूची जिन्हें डिफोल्ट रूप से निष्क्रिय किया गया है'}, {'en': 'Highlight duration', 'hi': 'अवधि को हाइलाइट रकें'}, {'en': 'The duration of the highlight box when selecting accessible nodes', 'hi': 'पहुंचनीय आसंधि (नोड) को चुनते समय हाइलाइट बक्से की अवधि'}, {'en': 'Highlight border color', 'hi': 'सीमांत (बोर्डर) के रंग को हाइलाइट करें'}, {'en': 'The color and opacity of the highlight border.', 'hi': 'हाइलाइट किए गए सीमांत का रंग और अपारदर्शिता। '}, {'en': 'Highlight fill color', 'hi': 'भराई के रंग

In [3]:
with open(r"D:\Documents\Deep_Learning\Generative-Adversarial-Networks\Transformers\data\quantum_mechanics_llm.txt", "r", encoding= 'utf-8') as f:
    text = f.read()
    
print(f"Length of dataset: {len(text)}")
print(text[:500])

Length of dataset: 5127

Quantum Mechanics: A Detailed Overview
--------------------------------------

Quantum Mechanics (QM) is the foundational theory of physics that describes nature at the smallest scales: atoms, subatomic particles, and photons. It is a paradigm-shifting framework that departs from classical mechanics and has deep implications not just in physics, but also in computation, information theory, and even machine learning.

1. Introduction to Quantum Mechanics
------------------------------------
At i


**2. Create a Vocabulary and Tokenizer**
We will use a character-level tokenizer as outlined in the PDF

In [4]:
chars = sorted(list(set(text)))
vocabs_size = len(chars)

stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for ch, i in stoi.items()}

def encode(s):
    return [stoi[c] for c in s]

def decode(l):
    return ''.join([itos[i] for i in l])

print(encode('Quantum'))
print(decode(encode('Quantum')))

[37, 70, 50, 63, 69, 70, 62]
Quantum


#### **3. Prepare the Dataset for training**
- Split into training and validation sets and generate inputs and outputs.


In [6]:
import torch

data = torch.tensor(encode(text), dtype= torch.long)
n  = int(0.9 * len(data))
train_data = data[0:n]
val_data = data[n:]

block_size = 128

def get_batch(split, batch_size= 64):
    data_split = train_data if split == 'train' else val_data
    ix = torch.randint(len(data_split) - block_size, (batch_size, ))
    x = torch.stack([data_split[i:i+block_size] for i in ix])
    y = torch.stack([data_split[i+1:i+1+block_size] for i in ix])
    return x, y


In [8]:
data.shape

torch.Size([5127])

In [9]:
x, y = get_batch("train")
print(x.shape, y.shape)

torch.Size([64, 128]) torch.Size([64, 128])


#### **4. Define a Simple LLM Model (Like GPT-Like Block)**
- Here a minimal transformer block model

In [11]:
import torch.nn as nn
import torch.nn.functional as F

class TinyTransformer(nn.Module):
    def __init__(self, vocab_size, n_embed):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, n_embed)
        self.fc = nn.Linear(n_embed, vocab_size)

    def forward(self, x, targets=None):
        x = self.embed(x)  # (B, T, C)
        logits = self.fc(x)  # (B, T, vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

#### **5. Train the Model**

In [12]:
model = TinyTransformer(vocabs_size, n_embed=128)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

for step in range(1000):
    xb, yb = get_batch("train")
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 100 == 0:
        print(f"Step {step}: loss = {loss.item():.4f}")


Step 0: loss = 4.8691
Step 100: loss = 2.4615
Step 200: loss = 2.2264
Step 300: loss = 2.1552
Step 400: loss = 2.1444
Step 500: loss = 2.1298
Step 600: loss = 2.1937
Step 700: loss = 2.1448
Step 800: loss = 2.1668
Step 900: loss = 2.1780


#### **6. Generate the text from the model**
- use the sampling to generate new quantum mechanics like text

In [13]:
def generate(model, start_text="Quantum", max_new_tokens=200):
    model.eval()
    context = torch.tensor(encode(start_text), dtype=torch.long)[None, :]  # (1, T)
    for _ in range(max_new_tokens):
        logits, _ = model(context)
        logits = logits[:, -1, :]
        probs = F.softmax(logits, dim=-1)
        next_id = torch.multinomial(probs, num_samples=1)
        context = torch.cat([context, next_id], dim=1)
    return decode(context[0].tolist())

print(generate(model))


Quantumprctont Dec  pes |0. Quaty ac /∂² mpant |
--
|^2.2
3 tio atiħ
--- pre, tity  cen asing dos. Fua at:
|1  LLL
-- past Ophesilen iounclesendanfonedongraun qung m----- D[x



-------  at) Than ----  |
|² 
